In [1]:
import duckdb
import logging
import pandas as pd
from pathlib import Path
from data.settings import settings
from data.alpha_vantage import AlphaVantageClient
from data.alpha_vantage_schema import ENDPOINT_TO_TABLE_MAP, TABLE_SCHEMAS, DEFAULT_ENDPOINTS

In [ ]:
# Configure logging
root_logger = logging.getLogger()
root_logger.setLevel(logging.INFO)

# Clear existing handlers to avoid duplicates/console pollution
if root_logger.handlers:
    for handler in root_logger.handlers:
        root_logger.removeHandler(handler)

# Ensure tables exist
db_path = Path(settings.get("data_dir"), settings.get("db_name"))
conn = duckdb.connect(str(db_path), read_only=False)
tables = set()

for endpoint_name in DEFAULT_ENDPOINTS:
   tables.add(ENDPOINT_TO_TABLE_MAP.get(endpoint_name, endpoint_name).upper())

for table_name in tables:
   schema_sql = TABLE_SCHEMAS.get(table_name)
   if schema_sql:
      create_sql = schema_sql.replace("CREATE TABLE", "CREATE TABLE IF NOT EXISTS")
      conn.execute(create_sql)

client = AlphaVantageClient(db_conn=conn)

data = client._fetch_data(
   endpoint_name="HISTORICAL_OPTIONS",
   params={"symbol": "GOOG", "date": "2011-11-24", "datatype": "csv"}
)
print(data)
# Update data efficiently
# client.smart_update(
#    symbols=['GOOG'],
#    endpoints=DEFAULT_ENDPOINTS,
#    start_date='2010-12-20',
#    end_date='2025-12-19'
# )
conn.close()

In [2]:
db_path = Path(settings.get("data_dir"), settings.get("db_name"))
conn = duckdb.connect(str(db_path), read_only=True)

result = conn.execute("SHOW TABLES").df()
print(result)

conn.close()

IOException: IO Error: Cannot open file "c:\users\roman\projects\alpha_vantage\data\alpha_vantage.db": The process cannot access the file because it is being used by another process.

File is already open in 
C:\Users\roman\miniconda3\envs\av\python.exe (PID 13108)